In [1]:
import pandas as pd
import numpy as np

In [47]:
!pip install fsspec
!pip install -U jupyter ipywidgets

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 121 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 243 kB 13.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 16.9 MB/s eta 0:00:01
     |████████████████████████████████| 119 kB 11.7 MB/s eta 0:00:01


     |████████████████████████████████| 54 kB 7.6 MB/s  eta 0:00:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [16]:
import tensorflow as  tf
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
import findspark
# os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3' #'/home/antslab/anaconda3/bin/python'
# findspark.init()
import pyspark
# from pyspark import SparkContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import Row, SQLContext, SparkSession, window
from pyspark import SparkConf, SparkContext
# Import data types
from pyspark.sql.types import *
import  pyspark.sql.functions as F
conf = SparkConf()
# conf.set("spark.local.dir", "/mnt/ssd240g/data/Leo_Spark_Home/tmp")
conf.set("spark.executor.cores","16")
conf.setMaster("local[16]")
conf.set("spark.driver.memory","200g") #滿載165G/
conf.set("spark.executor.memory", "95g")
conf.set("spark.memory.offHeap.enabled","true")
conf.set("spark.memory.offHeap.size","85g")
conf.set("spark.driver.extraJavaOptions","-Xss80g")
conf.set("spark.driver.extraJavaOptions","-Xms80g")
# conf.set("spark.memory.storageFraction","0.2")
# conf.set("spark.driver.maxResultSize","0")
# conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
# conf.set("spark.sql.execution.arrow.enabled", "true")
conf.set("spark.kubernetes.pyspark.pythonVersion","3")
conf.set("spark.network.timeout","2000s")
conf.set("spark.sql.shuffle.partitions",100000)
conf.set("spark.driver.maxResultSize", "150g")

# # spark = SparkSession.builder.appName("pcap_analyzer").config(conf=conf).getOrCreate() #pcap_analyzer pcapAnalyzer
# spark = SparkSession.builder.master("local").appName("Test_spark").config(conf=conf).getOrCreate()
# # spark.conf.set("spark.sql.execution.arrow.enabled", True)
# spark.conf.set("spark.sql.debug.maxToStringFields", 290000)
# spark.conf.set("spark.sql.hive.filesourcePartitionFileCacheSize",100*1024*1024*1024)

import databricks.koalas as ks
# ks.options.display.max_rows = 10
# ks.set_option('compute.max_rows', None)
# # ks.set_option('compute.ops_on_diff_frames', True)
# ks.set_option('compute.default_index_type', 'distributed')

/usr/bin/python
3.8.5 (default, Jan 27 2021, 15:41:15) 
[GCC 9.3.0]
sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)


In [16]:
time = '20200106'
isp = '台灣之星'
in_file = 'hdfs://192.168.50.200/user/hdfs/parquet/'+str(time[:4])+'_'+str(time[4:6])+'_'+str(time[6:])+'/'+isp+'/session_parquet/'+str(time)+'_session.parquet'
df = pd.read_parquet(in_file)

/usr/local/lib/python3.8/dist-packages/fsspec/implementations/hdfs.py:49: FutureWarning: pyarrow.hdfs.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  pahdfs = HadoopFileSystem(


FileNotFoundError: [Errno 2] No such file or directory: 'hadoop'

In [3]:
in_file

'hdfs://192.168.50.200/user/hdfs/parquet/2020_01_06/台灣之星/session_parquet/20200106_session.parquet'

In [44]:
import hdfs,pickle
from hdfs import InsecureClient
# hdfs_client = InsecureClient('http://192.168.50.200:9870', root='/')
hdfs_client = hdfs.Client("http://192.168.50.200:9870", root='/')
print(hdfs_client.list('/'))

['pcap_spark', 't.txt', 'user']


In [15]:
hdfs_client.download('./user/hdfs/parquet/2020_01_06/台灣之星/session_parquet/20200106_session.parquet')
/pcap_spark/2020_01_10/中華電信/parquet/2020_01_10_session_test.paruqet

TypeError: download() missing 1 required positional argument: 'local_path'

In [45]:
tmp = 'lkl'
hdfs_client.write('test.pkl', data=dumps(tmp), encoding='utf-8')

NameError: name 'dumps' is not defined

In [43]:
kk = hdfs_client.read('t.txt')
# for k in kk.gen:
kk

In [5]:
!pip install hdfs

  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=db412cd8378c37c2ab091dbe9f2d9bc279386f15e3350281e5da13445245c932
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


# transformer
* bert pooler output: [CLS] 經過linear和tanh得到(aka. pooling)，在BERT可能好、但在roberta可能不好
* hidden state output: 每個token的輸出，包含最前面的[CLS]和最後面的[SEP]，可以拿最後四層相加(每一層僅拿tokens自己做average)
* tokenizer可以自己先split (可以自己決定要留下那些符號、甚至可以先替換某些字詞)

In [84]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [98]:
encoded_input = tokenizer("this is Im a sample input") #[101, 8667, 146, 112, 182, 170, 1423, 5650, 102],
print(encoded_input)
encoded_input = tokenizer(["Hello", "I","'","m", "a", "single", "sentence"], is_split_into_words=True)
print(encoded_input)

{'input_ids': [101, 1142, 1110, 146, 1306, 170, 6876, 7758, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [101, 8667, 146, 112, 182, 170, 1423, 5650, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [30]:
print(len(encoded_input["input_ids"]))
tokenizer.decode(encoded_input["input_ids"])


8


'[CLS] Hello I m a single sentence [SEP]'

In [72]:
from transformers import TFBertModel, BertModel
model_tf = TFBertModel.from_pretrained('bert-base-cased')
input_tf = tokenizer("This is a sample input", return_tensors="tf")
output_tf = model_tf(input_tf)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [91]:
output_tf['last_hidden_state'][0][0]#.shape #pooler_output

<tf.Tensor: shape=(768,), dtype=float32, numpy=
array([-6.52061999e-02, -1.87395334e-01,  2.77693927e-01, -1.07218981e-01,
        2.39927620e-01, -1.33596331e-01,  5.33765137e-01, -1.37521252e-02,
       -2.32380658e-01, -4.14632291e-01,  2.58171856e-01,  4.05835547e-02,
       -1.81617230e-01,  2.71768510e-01, -4.02478337e-01,  6.65544271e-01,
       -2.47384682e-01, -4.26590294e-01, -1.14699043e-02, -9.64696765e-01,
        5.38871169e-01, -1.36399895e-01, -8.56562927e-02,  4.12345946e-01,
        6.60677671e-01,  2.91901499e-01,  7.21089303e-01,  2.81382620e-01,
       -4.56278443e-01, -2.55721986e-01,  1.15745209e-01,  6.99653566e-01,
       -3.52589667e-01, -2.85579383e-01, -2.93298155e-01,  2.50457853e-01,
       -1.96013480e-01, -6.90868974e-01, -1.47332624e-01, -8.20017219e-01,
       -4.72677648e-02,  5.53496659e-01,  2.41086215e-01, -5.67287385e-01,
       -1.74752459e-01, -3.87735605e-01, -4.63889614e-02,  1.25637874e-02,
       -4.33631241e-01, -4.87690046e-02,  8.82526755

In [92]:
output_tf['pooler_output'][0]

<tf.Tensor: shape=(768,), dtype=float32, numpy=
array([-0.6116774 ,  0.44734657,  0.9996649 , -0.99411094,  0.91647494,
        0.96336687,  0.98548377, -0.94078743, -0.97739226, -0.4327321 ,
        0.9946999 ,  0.9993347 , -0.9942997 , -0.998696  ,  0.75869906,
       -0.9831355 ,  0.99000794, -0.45526424, -0.999921  , -0.26178774,
       -0.19340879, -0.9995854 ,  0.25828528,  0.978599  ,  0.97049356,
       -0.01576787,  0.99562466,  0.9999361 ,  0.5580736 ,  0.621542  ,
        0.21360716, -0.99688494,  0.6202981 , -0.9990087 ,  0.31297642,
        0.13034703,  0.42679214, -0.271521  ,  0.18585421, -0.97260654,
       -0.42086717,  0.2828332 ,  0.06498186, -0.40839377,  0.9589422 ,
        0.42756137,  0.3977827 ,  0.03957479, -0.06858145,  0.99704224,
       -0.9498443 ,  0.9990143 , -0.95227087,  0.9980101 ,  0.9992048 ,
        0.68851316,  0.99880886,  0.12046936, -0.99229795,  0.5306975 ,
        0.9584186 ,  0.12580077,  0.9388689 , -0.40343282, -0.7092955 ,
       -0.518926

In [31]:
input_tf

{'input_ids': <tf.Tensor: shape=(1, 7), dtype=int32, numpy=array([[ 101, 1188, 1110,  170, 6876, 7758,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 7), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 7), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [ ]:
from transformers import TFAutoModel

In [71]:
model_tf.inputs

In [6]:
model_tf.summary()

Model: "tf_bert_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________


# pt bert

In [ ]:
model =  

## feature extract
* 可以用pipeline
    * 裡面可以讀取train好的model路徑
    * 可以指定pt/tf
    * 研究以batch一次大量(?)的用三張顯卡輸出，同時由CPU所有core將padding的vector拿掉再sum
* 可以記錄下每一句話他們tokenize以後有多長(因為會padding)，然後將各個token 對應的 output相加，再加上[CLS]的token vector，得到真正的packet embedding

In [55]:
from transformers import pipeline, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilroberta-base')
encoded_seq = tokenizer.encode("i am sentence","I am another")
feature_extraction = pipeline('feature-extraction', model="distilroberta-base", tokenizer=tokenizer,framework='tf')
features = feature_extraction(["i am sentence","I am another one two three"])
len(features[0])#,len(encoded_seq[0])

Some layers from the model checkpoint at distilroberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at distilroberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


8

In [59]:
features[1][-1]

[-0.040284302085638046,
 0.12525567412376404,
 -0.0523429773747921,
 -0.08992493897676468,
 0.12354927510023117,
 -0.11954562366008759,
 -0.01368824951350689,
 0.05975797399878502,
 0.01748603954911232,
 -0.06018880754709244,
 0.012487880885601044,
 0.04643506556749344,
 -0.0030959052965044975,
 -0.04444073140621185,
 0.0772448182106018,
 0.06105361878871918,
 0.001630924642086029,
 0.0071583460085093975,
 0.030082382261753082,
 0.059729911386966705,
 -0.10541784018278122,
 0.08233612030744553,
 -0.04365668445825577,
 0.12830345332622528,
 0.005045086145401001,
 0.03662016987800598,
 0.06580201536417007,
 0.06110576167702675,
 -0.012586385942995548,
 -0.03958055377006531,
 -0.1210668683052063,
 -0.0755377858877182,
 0.06804977357387543,
 0.0033936677500605583,
 0.0013847984373569489,
 0.014083858579397202,
 -0.009726117365062237,
 0.037351664155721664,
 -0.04918989911675453,
 0.05024796724319458,
 -0.07513990998268127,
 -0.009226389229297638,
 0.02794145978987217,
 0.016823414713144302

In [ ]:
np.array(features).shape

In [52]:
tokenizer.decode(encoded_seq)

'<s>i am sentence</s></s>I am another</s>'